# Text preprocessing for strategy score

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import re
import concurrent.futures

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install gcld3

     |████████████████████████████████| 3.8MB 5.7MB/s 


In [ ]:
import gcld3
detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=10000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MAX_THREADS = 30

In [ ]:
df_startups = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_deals.dta")
df_startups_unique = df_startups.drop_duplicates(["portfoliocompanyid"],keep="first")

df_public = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_public_firms.dta")
df_public_unique = df_public.drop_duplicates(["ÿþmark"],keep="first")

In [ ]:
drive_folder= "drive/My Drive/Capstone Shared Docs/result"
txt_folder = drive_folder + "/Txt Files Public and Startups"
txt_folder_2 = txt_folder.replace(" ","\ ")
!ls $txt_folder_2

'Public Companies'   Startups


In [ ]:
def get_text_paths(p):
  return list(p.glob('*.txt'))

def read_text_file(p):
  n = p.name.split("_")[0]
  file = open(p)
  # Remove first two and last characters, because of ascii encoding
  text = file.read().replace("***///***","\n")[2:-1]
  # remove ""
  text_list = text.split(" ")
  text_list = [x for x in text_list if x!=""]
  text = " ".join(text_list)
  file.close()
  return n,text

def get_text(files_list):
  threads = min(MAX_THREADS, len(files_list))
  company_text = {}
  data_futures = []
  with concurrent.futures.ThreadPoolExecutor(max_workers = threads) as thread_pool_executor:
    data_futures = [thread_pool_executor.submit(read_text_file, p) for p in files_list]
  for data_future in concurrent.futures.as_completed(data_futures):
      future = data_future.result()
      company_text[future[0]] = future[1]
  return company_text
    

def create_data_frame(company_text, company_type):
  company_frame = pd.DataFrame.from_dict(company_text, orient='index')
  company_frame.columns = ['text']
  # company_frame['companyid'] = company_frame.index
  # company_frame.index = np.arange(0,len(company_frame.companyid))
  company_frame = company_frame.assign(comp_type=company_type)
  stop_words_l=stopwords.words('english')
  company_frame['text_cleaned']=company_frame.text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
  company_frame['text_cleaned_with_sw']=company_frame.text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() ))
  company_frame["compid"] = company_frame.index
  company_frame.reset_index(inplace=True,drop=True)

  return company_frame

def get_path(c_type,year):
  if c_type=="S":
    folder_name = txt_folder + f"/Startups/Startup {year}"
  else:
    folder_name = txt_folder + f"/Public Companies/Public {year}"
  return pathlib.Path(folder_name)


def build_data_frame(c_type,year):
  path = get_path(c_type, year)
  files_path = get_text_paths(path)
  print(f"{c_type} {year} text files: "+str(len(files_path)))
  files = get_text(files_path)
  df = create_data_frame(files,c_type)
  return df

def merge_and_resolve_language(df1,df2):
  all_f = pd.concat([df1, df2]).reset_index(drop=True)
  
  # all_f.drop_duplicates(subset="text", keep="first",inplace=True)

  lan = []
  rel_lan = []
  for i,r in all_f.iterrows():
    t =r.text_cleaned
    t = str(t)
    
    result = detector.FindLanguage(text=t)
    lan.append(result.language)
    rel_lan.append(result.is_reliable)
  all_f['language'] = lan
  all_f['Rela_language'] = rel_lan

  # all_f_en = all_f[(all_f.language=='en') ].copy() #| ( (all_f.language!='en') & (all_f.language==False) )
  all_f_en = all_f
  # all_f_en.reset_index(drop=True,inplace=True)

  all_f_en["text_len"] = all_f_en.text_cleaned.apply(lambda x: len(x))
  all_f_en["words_len"] = all_f_en.text_cleaned.apply(lambda x: len(x.split()))

  return all_f_en

In [ ]:
def add_informative_columns(all_f, year):
  s_path = pathlib.Path(txt_folder +f"/Startups/Startup {year}")
  p_path = pathlib.Path(txt_folder +f"/Public Companies/Public {year}")

  startup_code_to_name = {}
  startup_code_to_website = {}
  public_code_to_name = {}
  public_code_to_website = {}

  for p in get_text_paths(s_path):
    n = int(p.name.split("_")[0])
    startup_code_to_name[n] = df_startups_unique[df_startups_unique.portfoliocompanyid==n].iloc[0].portfoliocompany
    startup_code_to_website[n] = df_startups_unique[df_startups_unique.portfoliocompanyid==n].iloc[0].website

  for p in get_text_paths(p_path):
    n = str(p.name.split("_")[0])
    public_code_to_name[n] = df_public_unique[df_public_unique["ÿþmark"]==n].iloc[0].companyname
    public_code_to_website[n] = df_public_unique[df_public_unique["ÿþmark"]==n].iloc[0].websiteaddress

  startup_code_to_info = df_startups_unique.set_index("portfoliocompanyid").background.to_dict()
  public_code_to_info = df_public_unique.set_index("ÿþmark").descriptionandhistory.to_dict()

  all_f.loc[all_f.comp_type=="S","website"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_website)
  all_f.loc[all_f.comp_type=="S","name"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_name)
  all_f.loc[all_f.comp_type=="S","info"] = all_f[all_f.comp_type=="S"].compid.astype(int).map(startup_code_to_info)

  all_f.loc[all_f.comp_type=="P","website"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_website)
  all_f.loc[all_f.comp_type=="P","name"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_name)
  all_f.loc[all_f.comp_type=="P","info"] = all_f[all_f.comp_type=="P"].compid.astype(str).map(public_code_to_info)

In [ ]:
def preprocess_year(year):
  df_1 = build_data_frame("S",year)
  df_2 = build_data_frame("P",year)
  df_3 = merge_and_resolve_language(df_1,df_2)
  add_informative_columns(df_3,year)
  return df_3

In [ ]:
# Run this to create csv for all years
for i in range(2011,2020):
  data = preprocess_year(i)
  data.to_csv(drive_folder+f"/complete_df/year_{i}.csv",index=False)
  print(f"Year {i} completed!")

S 2011 text files: 539
P 2011 text files: 8754
Year 2011 completed!
S 2012 text files: 769
P 2012 text files: 9204
